In [1]:
import os
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from google.colab import drive

# =========================================================
# 1. Colab 환경 설정 및 경로 정의 (필수)
# =========================================================
drive.mount('/content/drive')

# ⚠️ training.py에서 모델을 저장한 정확한 Google Drive 경로로 수정하세요!
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks'

# 모델 및 토크나이저가 저장된 폴더 경로
MODEL_DIR = os.path.join(PROJECT_DIR, "instruction_classifier_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# =========================================================
# 2. 모델 및 토크나이저 로드
# =========================================================
print(f"모델 로드 중: {MODEL_DIR}")
try:
    # ⚠️ AutoTokenizer 사용 및 로컬 파일 강제 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)

    # ⚠️ 로컬 파일 강제 로드
    model = BertForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
    model.to(device)
    model.eval()
    print("모델 로드 성공.")
except Exception as e:
    print(f"❌ 오류: 모델 로드 실패. 경로를 확인하거나 training.py 실행 여부를 확인하세요.")
    print(f"상세 오류: {e}")
    exit()

# =========================================================
# 3. 추론 함수 정의
# =========================================================
def is_instruction(sentence: str) -> bool:
    """
    주어진 문장이 지시문인지 (레이블 1) 아닌지 (레이블 0) 판별합니다.
    """
    # 1. 토큰화 및 인코딩
    encoding = tokenizer.encode_plus(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64,
        add_special_tokens=True, # KoBERT는 CLS, SEP 토큰 필요
        return_attention_mask=True
    )

    # 2. GPU로 데이터 이동
    input_ids = encoding["input_ids"].to(device)
    mask = encoding["attention_mask"].to(device)

    # 3. 모델 추론
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=mask)
        # 로짓(Logits)에서 가장 높은 값의 인덱스(레이블)를 추출
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # 지시문 레이블이 1이라고 가정하고 True/False 반환
    return prediction == 1

# =========================================================
# 4. 테스트
# =========================================================
print("\n--- 테스트 시작 ---")

# 지시문으로 예상되는 문장
test_sentence_instruction = "다음 구절의 의미를 파악해 보자."
result_instruction = is_instruction(test_sentence_instruction)
print(f"문장: \"{test_sentence_instruction}\" -> 지시문 여부: {result_instruction}")

# 지시문이 아닐 것으로 예상되는 문장 (예시: 평서문)
test_sentence_statement = "이것은 매우 중요한 개념입니다."
result_statement = is_instruction(test_sentence_statement)
print(f"문장: \"{test_sentence_statement}\" -> 지시문 여부: {result_statement}")

# 지시문이 아닐 것으로 예상되는 문장 (예시: 질문)
test_sentence_question = "이 문장이 지시문인가요?"
result_question = is_instruction(test_sentence_question)
print(f"문장: \"{test_sentence_question}\" -> 지시문 여부: {result_question}")

Mounted at /content/drive
사용 장치: cuda
모델 로드 중: /content/drive/MyDrive/Colab Notebooks/instruction_classifier_model
모델 로드 성공.

--- 테스트 시작 ---
문장: "다음 구절의 의미를 파악해 보자." -> 지시문 여부: True
문장: "이것은 매우 중요한 개념입니다." -> 지시문 여부: False
문장: "이 문장이 지시문인가요?" -> 지시문 여부: False


In [2]:
import os
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from google.colab import drive
import re # 문장 분리를 위해 정규표현식 라이브러리 추가

# =========================================================
# 1. Colab 환경 설정 및 경로 정의 (필수)
# =========================================================
drive.mount('/content/drive')

# ⚠️ training.py에서 모델을 저장한 정확한 Google Drive 경로로 수정하세요!
# 이 경로는 input_file.txt 파일이 위치한 경로와 동일해야 합니다.
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks'

# 모델 및 토크나이저가 저장된 폴더 경로
MODEL_DIR = os.path.join(PROJECT_DIR, "instruction_classifier_model")
INPUT_TXT_FILE = os.path.join(PROJECT_DIR, "inf_ex1.txt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# =========================================================
# 2. 모델 및 토크나이저 로드
# =========================================================
print(f"모델 로드 중: {MODEL_DIR}")
try:
    # ⚠️ AutoTokenizer 사용 및 로컬 파일 강제 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)

    # ⚠️ 로컬 파일 강제 로드
    model = BertForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
    model.to(device)
    model.eval()
    print("모델 로드 성공.")
except Exception as e:
    print(f"❌ 오류: 모델 로드 실패. 경로를 확인하거나 training.py 실행 여부를 확인하세요.")
    print(f"상세 오류: {e}")
    exit()

# =========================================================
# 3. 추론 함수 정의 (is_instruction)
# =========================================================
def is_instruction(sentence: str) -> bool:
    """
    주어진 문장이 지시문인지 (레이블 1) 아닌지 (레이블 0) 판별합니다.
    """
    # 문장이 너무 짧거나 공백이면 건너뜁니다.
    if not sentence or len(sentence.strip()) < 3:
        return False

    # 1. 토큰화 및 인코딩
    encoding = tokenizer.encode_plus(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64,
        add_special_tokens=True,
        return_attention_mask=True
    )

    # 2. GPU로 데이터 이동
    input_ids = encoding["input_ids"].to(device)
    mask = encoding["attention_mask"].to(device)

    # 3. 모델 추론
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=mask)
        # 로짓(Logits)에서 가장 높은 값의 인덱스(레이블)를 추출
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # 지시문 레이블이 1이라고 가정하고 True/False 반환
    return prediction == 1

# =========================================================
# 4. TXT 파일 처리 및 지시문 추출 함수
# =========================================================
def process_txt_file(file_path: str) -> list:
    """
    TXT 파일을 읽어 문장 단위로 분리하고, 지시문만 추출하여 반환합니다.
    """
    extracted_instructions = []

    print(f"\n--- 4. TXT 파일 처리 시작: {file_path} ---")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"❌ 오류: 입력 파일을 찾을 수 없습니다. 경로: {file_path}")
        return []
    except Exception as e:
        print(f"❌ 오류: 파일 읽기 중 오류 발생: {e}")
        return []

    # 텍스트를 문장 단위로 분리합니다. (마침표, 물음표, 느낌표 뒤에 공백이 오는 패턴 사용)
    # 한글 처리를 위해 복잡한 정규식 대신 일반적인 문장 부호와 re.split을 사용합니다.
    # Note: 정확한 한국어 문장 분리는 complex!
    sentences = re.split(r'([.?!])\s+', text)

    # re.split 결과는 문장과 구분 기호(., ?, !)가 교대로 나타나므로, 이를 다시 결합합니다.
    processed_sentences = []
    for i in range(0, len(sentences), 2):
        sentence = sentences[i]
        if i + 1 < len(sentences):
            # 문장과 구분 기호를 합칩니다.
            sentence += sentences[i+1]
        processed_sentences.append(sentence.strip())


    total_sentences = 0

    # 추출된 문장들을 모델에 넣어 판별합니다.
    for sentence in processed_sentences:
        if sentence:
            total_sentences += 1
            if is_instruction(sentence):
                extracted_instructions.append(sentence)

    print(f"총 문장 수: {total_sentences}")
    print(f"지시문으로 분류된 문장 수: {len(extracted_instructions)}")

    return extracted_instructions

# =========================================================
# 5. 실행
# =========================================================
'''
# 1. 테스트용 dummy TXT 파일 생성 (Colab 환경에서 테스트를 위해)
dummy_content = """
다음 문장을 읽어 보세요.
철수는 학교에 갔습니다.
이 구절의 핵심 내용을 요약해 보자.
오늘의 날씨는 맑을까요?
모든 객체의 속성을 변경하세요.
"""
with open(INPUT_TXT_FILE, 'w', encoding='utf-8') as f:
    f.write(dummy_content)
print(f"더미 입력 파일 생성 완료: {INPUT_TXT_FILE}")
'''

# 2. 파일 처리 및 지시문 추출
instructions = process_txt_file(INPUT_TXT_FILE)

# 3. 결과 출력
print("\n[ 최종 추출된 지시문 목록 ]")
if instructions:
    for i, instruction in enumerate(instructions):
        print(f"{i+1}. {instruction}")
else:
    print("추출된 지시문이 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
사용 장치: cuda
모델 로드 중: /content/drive/MyDrive/Colab Notebooks/instruction_classifier_model
모델 로드 성공.

--- 4. TXT 파일 처리 시작: /content/drive/MyDrive/Colab Notebooks/inf_ex1.txt ---
총 문장 수: 20
지시문으로 분류된 문장 수: 5

[ 최종 추출된 지시문 목록 ]
1. 이 시를 낭독해 보고, 운율을 형성하는 요소가 무엇인지 말해 보자.
2. ‘배려’를 주제로 한 시를 쓰고, 음악과 이미지를 넣어 영상으로 제작해 보자.
3. 3 1
1 에서 만든 영상 시를 사회 관계망 서비스(SNS)에 올려 친구들과 공유해 보자.
4. ● 내가 쓴 시
● 음악
시와 어울리는 음악이나 효과음을 넣어 볼까?
5. ♩♪
♬
● 이미지
시와 어울리는 사진이나 동영상을 찍어 보자.


In [3]:
import os
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from google.colab import drive
import re # 문장 분리를 위해 정규표현식 라이브러리 추가

# =========================================================
# 1. Colab 환경 설정 및 경로 정의 (필수)
# =========================================================
drive.mount('/content/drive')

# ⚠️ training.py에서 모델을 저장한 정확한 Google Drive 경로로 수정하세요!
# 이 경로는 input_file.txt 파일이 위치한 경로와 동일해야 합니다.
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks'

# 모델 및 토크나이저가 저장된 폴더 경로
MODEL_DIR = os.path.join(PROJECT_DIR, "instruction_classifier_model")
INPUT_TXT_FILE = os.path.join(PROJECT_DIR, "inf_ex2.txt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# =========================================================
# 2. 모델 및 토크나이저 로드
# =========================================================
print(f"모델 로드 중: {MODEL_DIR}")
try:
    # ⚠️ AutoTokenizer 사용 및 로컬 파일 강제 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)

    # ⚠️ 로컬 파일 강제 로드
    model = BertForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
    model.to(device)
    model.eval()
    print("모델 로드 성공.")
except Exception as e:
    print(f"❌ 오류: 모델 로드 실패. 경로를 확인하거나 training.py 실행 여부를 확인하세요.")
    print(f"상세 오류: {e}")
    exit()

# =========================================================
# 3. 추론 함수 정의 (is_instruction)
# =========================================================
def is_instruction(sentence: str) -> bool:
    """
    주어진 문장이 지시문인지 (레이블 1) 아닌지 (레이블 0) 판별합니다.
    """
    # 문장이 너무 짧거나 공백이면 건너뜁니다.
    if not sentence or len(sentence.strip()) < 3:
        return False

    # 1. 토큰화 및 인코딩
    encoding = tokenizer.encode_plus(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64,
        add_special_tokens=True,
        return_attention_mask=True
    )

    # 2. GPU로 데이터 이동
    input_ids = encoding["input_ids"].to(device)
    mask = encoding["attention_mask"].to(device)

    # 3. 모델 추론
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=mask)
        # 로짓(Logits)에서 가장 높은 값의 인덱스(레이블)를 추출
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # 지시문 레이블이 1이라고 가정하고 True/False 반환
    return prediction == 1

# =========================================================
# 4. TXT 파일 처리 및 지시문 추출 함수
# =========================================================
def process_txt_file(file_path: str) -> list:
    """
    TXT 파일을 읽어 문장 단위로 분리하고, 지시문만 추출하여 반환합니다.
    """
    extracted_instructions = []

    print(f"\n--- 4. TXT 파일 처리 시작: {file_path} ---")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"❌ 오류: 입력 파일을 찾을 수 없습니다. 경로: {file_path}")
        return []
    except Exception as e:
        print(f"❌ 오류: 파일 읽기 중 오류 발생: {e}")
        return []

    # 텍스트를 문장 단위로 분리합니다. (마침표, 물음표, 느낌표 뒤에 공백이 오는 패턴 사용)
    # 한글 처리를 위해 복잡한 정규식 대신 일반적인 문장 부호와 re.split을 사용합니다.
    # Note: 정확한 한국어 문장 분리는 complex!
    sentences = re.split(r'([.?!])\s+', text)

    # re.split 결과는 문장과 구분 기호(., ?, !)가 교대로 나타나므로, 이를 다시 결합합니다.
    processed_sentences = []
    for i in range(0, len(sentences), 2):
        sentence = sentences[i]
        if i + 1 < len(sentences):
            # 문장과 구분 기호를 합칩니다.
            sentence += sentences[i+1]
        processed_sentences.append(sentence.strip())


    total_sentences = 0

    # 추출된 문장들을 모델에 넣어 판별합니다.
    for sentence in processed_sentences:
        if sentence:
            total_sentences += 1
            if is_instruction(sentence):
                extracted_instructions.append(sentence)

    print(f"총 문장 수: {total_sentences}")
    print(f"지시문으로 분류된 문장 수: {len(extracted_instructions)}")

    return extracted_instructions

# =========================================================
# 5. 실행
# =========================================================
'''
# 1. 테스트용 dummy TXT 파일 생성 (Colab 환경에서 테스트를 위해)
dummy_content = """
다음 문장을 읽어 보세요.
철수는 학교에 갔습니다.
이 구절의 핵심 내용을 요약해 보자.
오늘의 날씨는 맑을까요?
모든 객체의 속성을 변경하세요.
"""
with open(INPUT_TXT_FILE, 'w', encoding='utf-8') as f:
    f.write(dummy_content)
print(f"더미 입력 파일 생성 완료: {INPUT_TXT_FILE}")
'''

# 2. 파일 처리 및 지시문 추출
instructions = process_txt_file(INPUT_TXT_FILE)

# 3. 결과 출력
print("\n[ 최종 추출된 지시문 목록 ]")
if instructions:
    for i, instruction in enumerate(instructions):
        print(f"{i+1}. {instruction}")
else:
    print("추출된 지시문이 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
사용 장치: cuda
모델 로드 중: /content/drive/MyDrive/Colab Notebooks/instruction_classifier_model
모델 로드 성공.

--- 4. TXT 파일 처리 시작: /content/drive/MyDrive/Colab Notebooks/inf_ex2.txt ---
총 문장 수: 48
지시문으로 분류된 문장 수: 10

[ 최종 추출된 지시문 목록 ]
1. 1 사건과 등장인물을 중심으로 이 작품의 내용을 정리해 보자.
2. ⑴ 주요 사건을 중심으로 빈칸에 들어갈 알맞은 말을 써 보자.
3. ⑶ 등장인물의 말과 행동을 보고, 인물의 성격과 처지를 파악해 보자.
4. ⑵ ⑴의 그림 카드 가 ~ 바 를 사건이 일어난 순서에 따라 배열해 보자.
5. 대화를 참고하여 작품 속 갈
등에 관한 자신의 생각을 말해 보자.
6. ● 점순이에게 맞서지 못해서 서럽고 화가 남.
7. ●점순이의 의도
‘나’의 반응
갈등의 진행
점순이가 ‘나’에게 감자를 주었으나, ‘나’가
.
8. ● 점순이가 또다시 닭싸움을 붙여 ‘나’의 닭이 공격당하
자 화가 나서
.
9. 한 권의 책을 읽으며 갈등을 파악해 보고
책 전시회를 열어 볼까?
10. 모
둠
⑵ 모둠에서 함께 읽을 책을 선정하고, 책을 선정한 까닭을 정리해 보자.


In [4]:
import os
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from google.colab import drive
import re # 문장 분리를 위해 정규표현식 라이브러리 추가

# =========================================================
# 1. Colab 환경 설정 및 경로 정의 (필수)
# =========================================================
drive.mount('/content/drive')

# ⚠️ training.py에서 모델을 저장한 정확한 Google Drive 경로로 수정하세요!
# 이 경로는 input_file.txt 파일이 위치한 경로와 동일해야 합니다.
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks'

# 모델 및 토크나이저가 저장된 폴더 경로
MODEL_DIR = os.path.join(PROJECT_DIR, "instruction_classifier_model")
INPUT_TXT_FILE = os.path.join(PROJECT_DIR, "inf_ex3.txt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# =========================================================
# 2. 모델 및 토크나이저 로드
# =========================================================
print(f"모델 로드 중: {MODEL_DIR}")
try:
    # ⚠️ AutoTokenizer 사용 및 로컬 파일 강제 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)

    # ⚠️ 로컬 파일 강제 로드
    model = BertForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
    model.to(device)
    model.eval()
    print("모델 로드 성공.")
except Exception as e:
    print(f"❌ 오류: 모델 로드 실패. 경로를 확인하거나 training.py 실행 여부를 확인하세요.")
    print(f"상세 오류: {e}")
    exit()

# =========================================================
# 3. 추론 함수 정의 (is_instruction)
# =========================================================
def is_instruction(sentence: str) -> bool:
    """
    주어진 문장이 지시문인지 (레이블 1) 아닌지 (레이블 0) 판별합니다.
    """
    # 문장이 너무 짧거나 공백이면 건너뜁니다.
    if not sentence or len(sentence.strip()) < 3:
        return False

    # 1. 토큰화 및 인코딩
    encoding = tokenizer.encode_plus(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64,
        add_special_tokens=True,
        return_attention_mask=True
    )

    # 2. GPU로 데이터 이동
    input_ids = encoding["input_ids"].to(device)
    mask = encoding["attention_mask"].to(device)

    # 3. 모델 추론
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=mask)
        # 로짓(Logits)에서 가장 높은 값의 인덱스(레이블)를 추출
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # 지시문 레이블이 1이라고 가정하고 True/False 반환
    return prediction == 1

# =========================================================
# 4. TXT 파일 처리 및 지시문 추출 함수
# =========================================================
def process_txt_file(file_path: str) -> list:
    """
    TXT 파일을 읽어 문장 단위로 분리하고, 지시문만 추출하여 반환합니다.
    """
    extracted_instructions = []

    print(f"\n--- 4. TXT 파일 처리 시작: {file_path} ---")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"❌ 오류: 입력 파일을 찾을 수 없습니다. 경로: {file_path}")
        return []
    except Exception as e:
        print(f"❌ 오류: 파일 읽기 중 오류 발생: {e}")
        return []

    # 텍스트를 문장 단위로 분리합니다. (마침표, 물음표, 느낌표 뒤에 공백이 오는 패턴 사용)
    # 한글 처리를 위해 복잡한 정규식 대신 일반적인 문장 부호와 re.split을 사용합니다.
    # Note: 정확한 한국어 문장 분리는 complex!
    sentences = re.split(r'([.?!])\s+', text)

    # re.split 결과는 문장과 구분 기호(., ?, !)가 교대로 나타나므로, 이를 다시 결합합니다.
    processed_sentences = []
    for i in range(0, len(sentences), 2):
        sentence = sentences[i]
        if i + 1 < len(sentences):
            # 문장과 구분 기호를 합칩니다.
            sentence += sentences[i+1]
        processed_sentences.append(sentence.strip())


    total_sentences = 0

    # 추출된 문장들을 모델에 넣어 판별합니다.
    for sentence in processed_sentences:
        if sentence:
            total_sentences += 1
            if is_instruction(sentence):
                extracted_instructions.append(sentence)

    print(f"총 문장 수: {total_sentences}")
    print(f"지시문으로 분류된 문장 수: {len(extracted_instructions)}")

    return extracted_instructions

# =========================================================
# 5. 실행
# =========================================================
'''
# 1. 테스트용 dummy TXT 파일 생성 (Colab 환경에서 테스트를 위해)
dummy_content = """
다음 문장을 읽어 보세요.
철수는 학교에 갔습니다.
이 구절의 핵심 내용을 요약해 보자.
오늘의 날씨는 맑을까요?
모든 객체의 속성을 변경하세요.
"""
with open(INPUT_TXT_FILE, 'w', encoding='utf-8') as f:
    f.write(dummy_content)
print(f"더미 입력 파일 생성 완료: {INPUT_TXT_FILE}")
'''

# 2. 파일 처리 및 지시문 추출
instructions = process_txt_file(INPUT_TXT_FILE)

# 3. 결과 출력
print("\n[ 최종 추출된 지시문 목록 ]")
if instructions:
    for i, instruction in enumerate(instructions):
        print(f"{i+1}. {instruction}")
else:
    print("추출된 지시문이 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
사용 장치: cuda
모델 로드 중: /content/drive/MyDrive/Colab Notebooks/instruction_classifier_model
모델 로드 성공.

--- 4. TXT 파일 처리 시작: /content/drive/MyDrive/Colab Notebooks/inf_ex3.txt ---
총 문장 수: 35
지시문으로 분류된 문장 수: 8

[ 최종 추출된 지시문 목록 ]
1. 이를 본 가람이가
포스터에서 보완해야 할 내용을 이야기해 주는데…….
2. (2)
1 나리가 처음에 만든 포스터와 수정한 포스터의 차이점을 말해 보자.
3. 2 나리와 같이 자신의 주장을 글로 써 본 경험을 이야기해 보자.
4. 너를 부회장으로 뽑아야
하는 타당한 근거를 들어 봐.
5. 너의 생각, 나의 의견

타당한 근거를 들어 주장하는 글은 어떻게 쓸까?
6. 정윤이가 발견한 문제 상황을 써 보자.
7. 개념 콕콕
정윤이와 친구들이 쓸 글의 주장과 이유, 예상 독자를 정리해 보자.
8. 그럼 우리가 쓸 글의 주장과 주장을 뒷받침하는 이유를 정리해 보자.
